## Obenhaus et al. - Figure S2 - Session level ratios of functional cell types

- Figure S2 H

In [1]:
# Set up basics
import sys, os
import numpy as np
import pandas as pd
from datetime import datetime
from matplotlib import pyplot as plt
import seaborn as sns

# Make plots pretty 
sns.set(style='white')
%config InlineBackend.figure_format = 'retina'

In [2]:
import datajoint as dj
sys.path.append('..')
from dj_schemas.utils import *
from dj_schemas.number_functional_celltypes import NumberCellTypes 

Connecting horsto@kavlidatajoint02.it.ntnu.no:3306


In [3]:
import dataframe_image as dfi
import cmasher as cmr

In [4]:
sys.path.append('..')
from dj_schemas.number_functional_celltypes import NumberCellTypes 

In [5]:
all_animals = np.array([
                    '82913','88592','87244','60480','97045','97046','89841', # Grid
                    '90222', '87245','87187','88106','94557','89622',        # OV
                    '90218','90647',                                         # other
                     ])
all_animals = all_animals.astype(int)
print(len(all_animals))

15


In [6]:
cell_results = Session.proj('animal_name') * (NumberCellTypes.MEC).proj(
                                                        mec_n_cells='mec_n_cells',
                                                        ratio_grid='mec_n_grid_95/mec_n_cells',
                                                        ratio_ov='mec_ovc/mec_n_cells',
                                                        ratio_info='mec_n_info_95/mec_n_cells',
                                                        ratio_hd='mec_n_hd_95/mec_n_cells', 
                                                        ratio_border='mec_n_border_95/mec_n_cells',            
                                                        ratio_bv='mec_n_bv_95/mec_n_cells',         
)

In [7]:
animal_col = ['animal_name']
number_col = ['mec_n_cells']
score_cols = ['ratio_grid','ratio_ov','ratio_hd','ratio_border','ratio_bv']
cols2keep  = animal_col + number_col + score_cols

In [8]:
cell_results_df = pd.DataFrame(cell_results.fetch(as_dict=True))
cell_results_df = cell_results_df[cols2keep]

In [9]:
cell_results_df = cell_results_df.apply(pd.to_numeric) # convert all columns of DataFrame

In [10]:
number_cell_results  = cell_results_df.groupby(by='animal_name').count()
grouped_cell_results = cell_results_df.groupby(by='animal_name').mean()
grouped_cell_results['session_count'] = number_cell_results.mec_n_cells # Just a random column which has the count()

# Sort by animal ...
grouped_cell_results = grouped_cell_results.loc[all_animals]

col_orderd = ['session_count'] + number_col + score_cols
grouped_cell_results = grouped_cell_results[col_orderd]

In [11]:
grouped_cell_results.head()

,session_count,mec_n_cells,ratio_grid,ratio_ov,ratio_hd,ratio_border,ratio_bv
animal_name,,,,,,,
82913,19,62.105263,0.102700,0.000000,0.167821,0.052305,0.054105
88592,10,132.300000,0.287920,0.004100,0.195110,0.096260,0.023960
87244,18,20.944444,0.240535,0.052111,0.130694,0.075165,0.041676
60480,6,84.833333,0.192517,0.058800,0.130117,0.067917,0.035517
97045,5,93.200000,0.193667,NaN,0.538233,0.030900,0.108833


In [12]:
renaming_dict = {
    'animal_name'   : 'Animal',
    'mec_n_cells'   : '# Cells',
    'session_count' : '# Sessions',
    'ratio_grid'    : 'Grid',
    'ratio_ov'      : 'OV',
    'ratio_hd'      : 'HD',
    'ratio_border'  : 'Border',
    'ratio_bv'      : 'BV',
}

In [13]:
grouped_cell_results = grouped_cell_results.reset_index().rename(columns = renaming_dict, inplace = False).set_index('Animal')

In [14]:
score_cols_A = ['Grid','OV']
score_cols_B = ['HD','Border','BV']

In [15]:
styled_df = grouped_cell_results.style \
                   .format({"# Cells": "{:.0f}"})\
                   .format({"# Sessions": "{:.0f}"})\
                   .format({"Grid": "{:.2f}"})\
                   .format({"OV": "{:.2f}"})\
                   .format({"HD": "{:.2f}"})\
                   .format({"Border": "{:.2f}"})\
                   .format({"BV": "{:.2f}"})\
                   .background_gradient(subset=score_cols_A, cmap='Greens', vmax=.3, vmin=0., axis=0) \
                   .bar(subset='# Cells', vmin=0, vmax=220, color='#ccc' )\
                   .bar(subset='# Sessions', vmin=0, vmax=20, color='#ccc' )

In [16]:
styled_df 

,# Sessions,# Cells,Grid,OV,HD,Border,BV
Animal,,,,,,,
82913,19,62,0.10,0.00,0.17,0.05,0.05
88592,10,132,0.29,0.00,0.20,0.10,0.02
87244,18,21,0.24,0.05,0.13,0.08,0.04
60480,6,85,0.19,0.06,0.13,0.07,0.04
97045,5,93,0.19,nan,0.54,0.03,0.11
97046,6,195,0.21,0.10,0.48,0.10,0.16
89841,4,70,0.16,0.00,0.19,0.07,0.03
90222,11,183,0.03,0.13,0.28,0.09,0.14
87245,7,187,0.06,0.14,0.25,0.12,0.09


In [18]:
# If you want to export 
#dfi.export(styled_df, 'df_styled.png', fontsize=80)